# Interpolation to Spherical Grids in the Einstein Toolkit


<font color='blue'>**This module has not yet undergone validation testing.**</font>

## Structure

This module is designed to interpolate arbitrary quantities on [Einstein Toolkit](https://einsteintoolkit.org/) Adaptive-Mesh Refinement (AMR) grids (using the [Carpet](https://carpetcode.org/) AMR infrastructure) to numerical grids with spherical sampling.

Given some set of $N$ quantities $\mathbf{Q}=\{Q_0,Q_1,Q_2,...,Q_{N-2},Q_{N-1}\}$, this module performs the following for each $Q_i$:

1. Evaluate $Q_i$ at all gridpoints that are not ghost zones. Sometimes $Q_i$ is computed using finite difference derivatives, so this is necessary.
1. Call upon Carpet's interpolation and interprocessor synchronization functions to fill in $Q_i$ at all ghost zones, *except* at the outer boundary. We do not generally trust $Q_i$ at the outer boundary due to errors associated with the approximate outer boundary conditions. 
1. At this point, $Q_i$ is set at all gridpoints except ghost zones at the outer boundary. Interpolate $Q_i$ to the spherical grids, **maintaining the Cartesian basis for all vectors and tensors**, and append the result to a file.

In [1]:
!mkdir interp_sph_grids_ETK     2>/dev/null # 2>/dev/null: Don't throw an error or warning if the directory already exists.
!mkdir interp_sph_grids_ETK/src 2>/dev/null # 2>/dev/null: Don't throw an error or warning if the directory already exists.

### Low-Level ETK Interpolation Function

We start by writing the low-level interpolation function **Interpolate_to_sph_grid()**, which  to file. 

**Interpolate_to_sph_grid()** takes as input
* **cctkGH**: Information about the underlying Cactus/Carpet grid hierarchy.
* **interp_num_points**: Number of destination interpolation points
* **point_x_temp, point_y_temp, point_z_temp**: Cartesian $(x,y,z)$ location for each of the **interp_num_points** interpolation points.
* **input_array_names[1]**: List of input gridfunction names to interpolate. We will do this only one gridfunction at a time, for gridfunction $Q_i$, as described above.

**Interpolate_to_sph_grid()** outputs:
* **output_f[1]**: The gridfunction **input_array_names[1]** interpolated to the set of **interp_num_points** specified in the input.

In [2]:
%%writefile interp_sph_grids_ETK/src/Interpolate_to_sph_grid.h

void Interpolate_to_sph_grid(cGH *cctkGH,CCTK_INT interp_num_points,
                             CCTK_REAL *point_x_temp,CCTK_REAL *point_y_temp,CCTK_REAL *point_z_temp, 
                             const CCTK_STRING input_array_names[1], CCTK_REAL *output_f[1]) {
  DECLARE_CCTK_PARAMETERS; 
  CCTK_INT ierr;

  const CCTK_INT NUM_INPUT_ARRAYS=1;
  const CCTK_INT NUM_OUTPUT_ARRAYS=1;

  CCTK_STRING coord_system = "cart3d";

  // Set up handles
  const CCTK_INT coord_system_handle = CCTK_CoordSystemHandle(coord_system);
  if (coord_system_handle < 0) {
    CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
        "can't get coordinate system handle for coordinate system \"%s\"!",
               coord_system);
  }

  const CCTK_INT operator_handle = CCTK_InterpHandle(interpolator_name);
  if (operator_handle < 0)
    CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
               "couldn't find interpolator \"%s\"!",
               interpolator_name);

  CCTK_INT param_table_handle = Util_TableCreateFromString(interpolator_pars);
  if (param_table_handle < 0) {
    CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
               "bad interpolator parameter(s) \"%s\"!",
               interpolator_pars);
  }
  
  CCTK_INT operand_indices[NUM_INPUT_ARRAYS]; //NUM_OUTPUT_ARRAYS + MAX_NUMBER_EXTRAS];
  for(int i = 0 ; i < NUM_INPUT_ARRAYS  ; i++) {
    operand_indices[i] = i;
  }
  Util_TableSetIntArray(param_table_handle, NUM_OUTPUT_ARRAYS,
                        operand_indices, "operand_indices");
  

  CCTK_INT opcodes[NUM_INPUT_ARRAYS];
  for(int i = 0 ; i < NUM_INPUT_ARRAYS  ; i++) {
    opcodes[i] = 0;
  }
  Util_TableSetIntArray(param_table_handle, NUM_OUTPUT_ARRAYS, 
                        opcodes, "opcodes");

  const void* interp_coords[3] 
    = { (const void *) point_x_temp,
        (const void *) point_y_temp,
        (const void *) point_z_temp };

  CCTK_INT input_array_indices[NUM_INPUT_ARRAYS];
  for(int i = 0 ; i < NUM_INPUT_ARRAYS ; i++) {
    input_array_indices[i] = CCTK_VarIndex(input_array_names[i]);
    if(input_array_indices[i] < 0) {
      CCTK_VWarn(0, __LINE__, __FILE__, CCTK_THORNSTRING,
        "COULD NOT FIND VARIABLE '%s'.",
        input_array_names[i]);
      exit(1);
    }
  }

  CCTK_INT output_array_types[NUM_OUTPUT_ARRAYS];
  for(int i = 0 ; i < NUM_OUTPUT_ARRAYS ; i++) {
    output_array_types[i] = CCTK_VARIABLE_REAL;
  }

  void * output_arrays[NUM_OUTPUT_ARRAYS]
    = { (void *) output_f[0] };

  // actual interpolation call
  ierr = CCTK_InterpGridArrays(cctkGH,
                               3, // number of dimensions 
                               operator_handle,
                               param_table_handle,
                               coord_system_handle,
                               interp_num_points,
                               CCTK_VARIABLE_REAL,
                               interp_coords,
                               NUM_INPUT_ARRAYS, // Number of input arrays
                               input_array_indices,
                               NUM_OUTPUT_ARRAYS, // Number of output arrays
                               output_array_types,
                               output_arrays);
  if (ierr<0) {
    CCTK_WARN(1,"interpolation screwed up");
    Util_TableDestroy(param_table_handle);
    exit(1);
  }

  ierr = Util_TableDestroy(param_table_handle);
  if (ierr != 0) {
    CCTK_WARN(1,"Could not destroy table");
    exit(1);
  }
}

Overwriting interp_sph_grids_ETK/src/Interpolate_to_sph_grid.h


### Setting up the Spherical Grids


* By default, we set logarithmic radial coordinates: $r(x_{0,i}) = R_0 + e^{x_{0,i}}$, where
  + $x_{0,i} = x_{0, \mathrm{beg}} + \left(i+\frac{1}{2}\right) \Delta x_0$
  + $x_{0, {\mathrm{beg}}} = \log\left( R_{\mathrm{in}} - R_0 \right)$
  + $\Delta x_0 = \frac{1}{N_0}\log\left(\frac{R_\mathrm{out} - R_0}{R_\mathrm{in} - R_0}\right)$
* As for the polar angle $\theta$, there are two options:
  + **Option 1**: 
  $$
  \theta(x_{1,j})  \, = \, \theta_c \, + \, \left( \pi - 2 \theta_c \right) x_{1,j} \, 
+ \, \xi \, \sin\left(2 \pi x_{1,j} \right) \text{, where}
$$
    + $x_{1,j} = x_{1, \mathrm{beg}} + \left(j+\frac{1}{2}\right) \Delta x_1$
    + $\Delta x_1 = \frac{1}{N_1}$
  + **Option 2**: 
  $$
  \theta(x_{1,j}) = \frac{\pi}{2} \left[  1  + \left(1-\xi \right) \left(2 x_{1,j} - 1 \right) + \left( \xi - \frac{2 \theta_c}{\pi} \right) \left( 2 x_{1,j} - 1 \right)^n \right] \text{, where}
  $$
    + $n$ is odd
    + $x_{1,j} = x_{1, \mathrm{beg}} + \left(j+\frac{1}{2}\right) \Delta x_1$
    + $\Delta x_1 = \frac{1}{N_1}$
* The azimuthal angle $\phi$ is uniform, so that $\phi(x_{2,k}) = x_{2,k}$:
  + $x_{2,k} \in [0,2\pi]$
  + $x_{2,k} = x_{2, \mathrm{beg}} + \left(k+\frac{1}{2}\right)\Delta x_{2}$
  + $\Delta x_{2} = \frac{ 2 \pi }{N_2}$

In [3]:
%%writefile interp_sph_grids_ETK/src/Set_up_interp_points_on_sph_grid.h

void sph_grid_Interpolate_many_pts__set_interp_pts(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  CCTK_REAL dx0 = log( (Rout - R0) / (Rin - R0) ) / ((CCTK_REAL)N0);
  CCTK_REAL dx1 = 1.0 / ((CCTK_REAL)N1);
  CCTK_REAL dx2 = 1.0 / ((CCTK_REAL)N2);

  CCTK_INT which_pt = 0;
  for(CCTK_INT k=0;k<N2;k++) for(CCTK_INT j=0;j<N1;j++) for(CCTK_INT i=0;i<N0;i++) {
    CCTK_REAL x0_i = x0_beg + ((CCTK_REAL)i + 0.5)*dx0;
    CCTK_REAL rr = R0 + exp(x0_i);

    CCTK_REAL x1_j = x1_beg + ((CCTK_REAL)j + 0.5)*dx1;
    CCTK_REAL th = -1e300;
    if(theta_option == 1) {
       th = th_c + (M_PI - 2.0*th_c)*x1_j + xi*sin(2.0*M_PI*x1_j);
    } else if (theta_option == 2) {
       th = M_PI/2.0 * ( 1.0 + (1.0 - xi)*(2.0*x1_j - 1.0) + (xi - 2.0*th_c/M_PI)*pow(2.0*x1_j - 1.0 ,th_n) );
    } else {
       printf("Error: theta_option = %d NOT SUPPORTED.",theta_option);
       exit(1);
    }
    
    CCTK_REAL x2_k = x2_beg + ((CCTK_REAL)k + 0.5)*dx2;
    CCTK_REAL ph = x2_k;

    points_x[which_pt] = rr*sin(th)*cos(ph);
    points_y[which_pt] = rr*sin(th)*sin(ph);
    points_z[which_pt] = rr*cos(th);
    which_pt++;
  }
}

Overwriting interp_sph_grids_ETK/src/Set_up_interp_points_on_sph_grid.h


### Outputting to File

Since they take almost no space, we attach the entire metadata to each interpolated function that is output:

In [4]:
%%writefile interp_sph_grids_ETK/src/output_to_file.h

void output_to_file(CCTK_ARGUMENTS,CCTK_REAL *output_f[1]) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  char filename[100];
  sprintf (filename, "%s/interp_sph_grids.dat", out_dir);
  FILE *file = fopen (filename,"a+");
  if (! file) {
    CCTK_VWarn (1, __LINE__, __FILE__, CCTK_THORNSTRING,
                "interp_sph_grid__ET_thorn: Cannot open output file '%s'", filename);
    exit(1);
  }

  fwrite(&N0, sizeof(CCTK_INT), 1, file);
  fwrite(&x0_beg, sizeof(CCTK_REAL), 1, file);
  fwrite(&R0, sizeof(CCTK_REAL), 1, file);
  fwrite(&Rin, sizeof(CCTK_REAL), 1, file);
  fwrite(&Rout, sizeof(CCTK_REAL), 1, file);

  fwrite(&N1, sizeof(CCTK_INT), 1, file);
  fwrite(&x1_beg, sizeof(CCTK_REAL), 1, file);
  fwrite(&theta_option, sizeof(CCTK_INT), 1, file);
  fwrite(&th_c, sizeof(CCTK_REAL), 1, file);
  fwrite(&xi, sizeof(CCTK_REAL), 1, file);
  fwrite(&th_n, sizeof(CCTK_INT), 1, file);

  fwrite(&N2, sizeof(CCTK_INT), 1, file);
  fwrite(&x2_beg, sizeof(CCTK_REAL), 1, file);

  CCTK_REAL magic_number = 1.130814081305130e-21;
  fwrite(&magic_number, sizeof(CCTK_REAL), 1, file);
  fwrite(&cctk_iteration, sizeof(CCTK_INT), 1, file);
  fwrite(&cctk_time, sizeof(CCTK_REAL), 1, file);
  for(CCTK_INT i=0;i<1;i++) {
    fwrite(output_f[i], sizeof(CCTK_REAL)*N0*N1*N2, 1, file);
  }

  fclose(file);
}

Overwriting interp_sph_grids_ETK/src/output_to_file.h


## The Main Interpolation Driver Function

The **Interpolate_to_sph_grid_main_function()** function calls the above functions as follows:
1. **sph_grid_Interpolate_many_pts__set_interp_pts()**: First set up the spherical grids
1. **Interpolate_to_sph_grid()**: Output

In [5]:
%%writefile interp_sph_grids_ETK/src/main_function.cc

// Include needed ETK & C library header files:
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <ctype.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "cctk_Functions.h"
#include "util_Table.h"
#include "util_String.h"

// Include locally-defined C++ functions:
#include "Set_up_interp_points_on_sph_grid.h"
#include "Interpolate_to_sph_grid.h"
#include "output_to_file.h"

void Interpolate_to_sph_grid_main_function(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // Perform interpolation only at iteration == interp_out_iteration:
  if(cctk_iteration != interp_out_iteration) return;

  // Set up spherically sampled interpolation grid arrays points_x,points_y,points_z: 
  sph_grid_Interpolate_many_pts__set_interp_pts(CCTK_PASS_CTOC);

    
    
  // Set up output array:
  CCTK_REAL *output_f[1];
  output_f[0] = output_interped;
  // The name of the input gridfunction is always "interp_sph_grids_ETK::interped_gf":
  const CCTK_STRING input_array_names[1] = { "interp_sph_grids_ETK::interped_gf" };

  // Perform interpolation!
  Interpolate_to_sph_grid(cctkGH, interp_num_points,
                             points_x,points_y,points_z, input_array_names, output_f);

  if(CCTK_MyProc(cctkGH)==0) {
    output_to_file(CCTK_PASS_CTOC,output_f);
    printf("Interpolate_to_sph_grid_main_function(): Just output to file at iteration %d\n",cctk_iteration);
  } else {
    printf("Interpolate_to_sph_grid_main_function(): Process !=0 waiting for file output at iteration %d\n",cctk_iteration);
  }
}

Overwriting interp_sph_grids_ETK/src/main_function.cc


### Interpolation Function Counter Routines, for Scheduling

In [6]:
%%writefile interp_sph_grids_ETK/src/interp_counter.cc

void SphGrid_InitializeInterpCounterToZero(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  *InterpCounter = 0;

  if(verbose==2) printf("interp_sph_grids_ETK: Just set InterpCounter to %d\n",*InterpCounter);
}

void SphGrid_GRMHD_InitializeInterpCounter(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  if(cctk_iteration%VolIntegral_out_every==0) {
    *InterpCounter = NumInterpFunctions;
    if(verbose==2) printf("interp_sph_grids_ETK: Just set InterpCounter to %d == NumInterpFunctions\n",
                          *InterpCounter);
  }
}

void SphGrid_GRMHD_DecrementInterpCounter(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

    (*InterpCounter)--;
    if(verbose==2) printf("interp_sph_grids_ETK: Just decremented InterpCounter to %d\n",*InterpCounter);
}

Overwriting interp_sph_grids_ETK/src/interp_counter.cc


# Use NRPy+ C Output to Set All Output Gridfunctions


In [7]:
# Step 1: Import needed NRPy+ parameters
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import sympy as sp
import NRPy_param_funcs as par
import loop

from collections import namedtuple
gf_interp = namedtuple('gf_interp', 'name interp_order')
gf_interp_list = []

gf_interp_list.append(gf_interp("IllinoisGRMHD::rho_b","1"))

#interp_expr = 

In [8]:
%%writefile interp_sph_grids_ETK/src/interp_functions.cc

// Set the gridfunction interped_gf, according to the interpolation counter variable interp_counter.
//    For example, we might interpolate "IllinoisGRMHD::rho_b" if interp_counter==0. The following
//    function takes care of these


Overwriting interp_sph_grids_ETK/src/interp_functions.cc
